Esse script acessa os dados do Lieden Ranking e gera arquivos json para a visualização de dados montada a partir da biblioteca `Highcharts`.

In [1]:
import json
import pandas as pd
import random

Primeiro, precisamos ler o arquivo original.

In [2]:
def read_excel(fp, sheet):
    '''
    Lê uma planilha, especificada pelo nome 'sheets',
    do arquivo xls especificado em 'fp'.
    '''

    with pd.ExcelFile(fp) as xls:
        df = pd.read_excel(xls, sheet)
        
    return df

A função abaixo separa a string que define um intervalo temporal em duas colunas com valores do tipo `int`.

In [3]:
def split_years(row):
    '''
    Cria colunas contendo o ano final
    e o ano inicial da coluna 'Period'
    '''
    
    years = row.Period.split("–")
    
    Period_start = years[0]
    Period_end   = years[1]
    
    return pd.Series({
        "Period_start" : int(Period_start),
        "Period_end"   : int(Period_end)
    })

As funções abaixo renomeiam alguns campos para que a visualização fique consistente em linguagem e padronização.

In [4]:
def rename_field(field):
    '''
    Altera o padrão de nomenclatura 
    dos campos de conhecimento.
    '''

    corresp = {
        "All sciences" : "Todos os campos",
        "Biomedical and health sciences" : "Saúde",
        "Life and earth sciences" : "Biológicas",
        "Physical sciences and engineering" : "Física",
        "Social sciences and humanities" : "Humanidades",
        "Mathematics and computer science" : "Matemática"
    }

    return corresp[field]

In [5]:
def rename_univ(univ):
    '''
    Altera o padrão de nomenclatura
    das universidades brasileiras.
    '''

    corresp = {
         'Federal University of Paraíba': 'UFPB – Univ. Fed. da Paraíba',
         'Federal University of Pernambuco': 'UFPE – Univ. Fed. de Pernambuco',
         'Federal University of Rio Grande do Norte': 'UFRN – Univ. Fed. do Rio Grande do Norte',
         'Federal University of Rio Grande do Sul': 'UFRGS – Univ. Fed. do Rio Grande do Sul',
         'Federal University of Santa Maria': 'UFSM – Univ. Fed. de Santa Maria',
         'Federal University of São Carlos': 'Ufscar – Univ. Fed. de São Carlos',
         'Federal University of Uberlândia': 'UFU – Univ. Fed. de Uberlândia',
         'Rio de Janeiro State University': 'UERJ – Univ. Est. do Rio de Janeiro',
         'State University of Maringá': 'UEM – Univ. Est. de Maringá',
         'Universidade Estadual Paulista': 'Unesp – Univ. Est. Paulista',
         'Universidade Federal Fluminense': 'UFF – Univ. Fed. Fluminense ',
         'Universidade Federal da Bahia': 'UFBA – Univ. Fed. da Bahia',
         'Universidade Federal de Goiás': 'UFGO – Univ. Fed. de Goiás',
         'Universidade Federal de Minas Gerais': 'UFMG – Univ. Fed. de Minas Gerais',
         'Universidade Federal de Santa Catarina': 'UFSC – Univ. Fed. de Santa Catarina',
         'Universidade Federal de São Paulo': 'Unifesp – Univ. Fed. de São Paulo',
         'Universidade Federal de Viçosa': 'UFV – Univ. Fed. de Viçosa',
         'Universidade Federal do Ceará': 'UFC – Univ. Fed. do Ceará',
         'Universidade Federal do Paraná': 'UFPR - Univ. Fed. do Paraná',
         'Universidade Federal do Rio de Janeiro': 'UFRJ – Univ. Fed. do Rio de Janeiro',
         'Universidade de Brasília': 'UnB – Univ. de Brasília',
         'University of Campinas': 'Unicamp – Univ. Est. de Campinas',
         'University of São Paulo': 'USP – Univ. de São Paulo'
    }

    return corresp[univ]

Abaixo, geramos uma cor hexadecimal aleatória. Isso serve apenas para facilitar nosso trabalho no Adobe Illustrator

In [6]:
def randomize_color():
    r = lambda: random.randint(0,255)
    return ('#%02X%02X%02X' % (r(),r(),r()))

Agora, a função para extrair os dados do csv e montar um arquivo json com universidades ordendas por uma métrica de resultado.

In [12]:
def extract_top_univs(df, country, period_end, n_items, ordering_col, filename, color = "#FFB6C1"):
    '''
    A função seleciona as 'n_items' universidades com maior produtividade no país e 
    período selecionados, sempre considerando todas as áreas do conhecimento possíveis. 
    
    A variável 'ordering_col' deve ser usada para determinar qual coluna será usada para mensurar 
    produtividade. Em geral, usamos 'impact_P' para mensurar o total de artigos publicados, 'P_top10'
    para mensurar o total de artigos publicados entre os 5% de maior impoacto no mundo, e 'PP_top10'
    para mensurar o percentual de artigos publicados pela universidade que estão entre estes 5%.
    
    Depois de fazer essa seleção, a função recupera também o indicador passado em 'ordering_col'
    para todos os campos do conhecimento das universidades selecionados.
    
    É possível selecionar uma cor customizada para o json, também.
    
    O arquivo é salvo no diretório de visualização usando o nome de arquivo estipulado em 'filename'
    
    Abaixo, um exemplo de output:
    
    >>> extract_top_univs(df = df, 
                          country = "Brazil",
                          period_end = 2017,
                          n_items = 2,
                          ordering_col = 'impact_P', 
                          filename = 'exemplo.csv')
    
    >>> ../viz/data/exemplo.json
    
    series : [
     {
       color: "FFB61",
       data: [
           { name : "SAÚDE", y : 8391 },
           { name : "BIOLÓGICAS", y : 3703 },
           { name : "MATEMÁTICA", y : 1060},
           { name : "FÍSICA", y : 3248},
           { name : "HUMANIDADES, y : 444 }
       ],
       name: "USP",
       type: "areaspline"
     },
    {
       color: "FFB61",
       data: [
           { name : "SAÚDE", y : 839 },
           { name : "BIOLÓGICAS", y : 373 },
           { name : "MATEMÁTICA", y : 160},
           { name : "FÍSICA", y : 348},
           { name : "HUMANIDADES, y : 41 }
       ],
       name: "UFXYZ",
       type: "areaspline"
     }
    ]
    '''

    # 1. Seleciona e ordena as 'n_items' universidades pelo valor selecionado em 'ordering_col'
    univs = df [ 
               (df.Country == country) &  
               (df.Frac_counting == 1) &
               (df.Field == "All sciences") &
               (df.Period_end == period_end)
              ]
        
    univs = univs[ [ "University", ordering_col, "Field" ] ]    
        
    univs = univs.sort_values(by = ordering_col, ascending = False)
        
    univs = univs.head(n_items).University.tolist()
    
    # 2. Cria o objeto JSON e calcula os valores para todos os campos
    json_obj = { "series" : [ ] }
    
    fields = [ field for field in df.Field.unique() if field != "All sciences"]
    
    if color == 'random':
        used_colors = [ ]
        
    for univ in univs:
        
        # Objeto com dados da universidade específica
        # alguns parâmetros universais
        inner_obj = { }
        
        if color == 'random':
            
            this_color = ''
            while True:
                
                this_color = randomize_color()
                
                if this_color in used_colors:
                    this_color = randomize_color()
                    
                else:
                    used_colors.append(this_color)
                    break
                    
        else:
            this_color = color
            
        inner_obj["color"] = this_color
        inner_obj["type"] = "areaspline"
        inner_obj["name"] = rename_univ(univ)
        inner_obj["data"] = [ ]

        for field in fields:
            
            # Objeto para guardar os números de cada campo
            field_data = { }
            field_data["name"] = rename_field(field).upper()

            # Seleciona os dados da universidade e campo desejados
            temp = df [              
                (df.University == univ) &
                (df.Country == country) & 
                (df.Field == field ) &
                (df.Frac_counting == 1) &
                (df.Period_end == period_end)
              ]

            assert temp.shape[0] == 1
            temp = temp.reset_index()

            field_data["y"] = round(temp.loc[0, ordering_col], 4)

            inner_obj["data"].append(field_data)
                
        json_obj["series"].append(inner_obj)
        
    # Salva o JSON
    with open(f"../viz/data/{filename}.json", "w+") as file:
        json.dump(json_obj, file, indent = 2)
        
            
            
            


Aqui, uma função que cria um json com o agregado de toda a produção cientítica do Brasil. É uma adaptação simples da função acima.

In [13]:
def get_aggregate(df, country, period_end, measure, filename, opp = 'sum', color = "#FFB6C1"):

    # 1. Seleciona o país e faz as somas
    country_df = df [ 
               (df.Country == country) &  
               (df.Frac_counting == 1) &
               (df.Period_end == period_end) &
               (df.Field != "All sciences")
              ]
        
    country_df = country_df[ [ "University", measure, "Field" ] ] 
    
    # Cria o objeto json
    json_obj = { "series" : [ ] }
    
    fields = [ field for field in country_df.Field.unique() if field != "All sciences"]
            
    inner_obj = { }
    inner_obj["color"] = color
    inner_obj["type"] = "areaspline"
    inner_obj["name"] = country
    inner_obj["data"] = [ ]
    
    # Faz as somas necessárias
    for field in fields:
                    
        if opp == 'sum':
            field_data = {
                "name" : rename_field(field).upper(),
                "y" : df [ df.Field == field][measure].sum()
            }
        
        elif opp == 'mean':
            field_data = {
                "name" : rename_field(field),
                "y" : df [ df.Field == field][measure].mean()
            }

        inner_obj["data"].append(field_data)
    
    json_obj["series"].append(inner_obj)
        
    # Salva o JSON
    with open(f"../viz/data/{filename}.json", "w+") as file:
        json.dump(json_obj, file, indent = 2)

Enfim, precisamos gerar um arquivo CSV com coordenadas X e Y para posicionar os dados em um scatter plot.

In [14]:
def get_xy(df, x_col, y_col, period_end, country, filename):
    
    data = df [ 
               (df.Country == country) &  
               (df.Frac_counting == 1) &
               (df.Period_end == period_end) &
               (df.Field == "All sciences")
              ]

    data = data [ ["University", x_col, y_col ]]
    
    def change_univ(row):
        university = rename_univ(row.University)
        return pd.Series({"University" : university})
    
    data["University"] = data.apply(change_univ, axis = 1)
    
    data.to_csv(f"../viz/data/{filename}.csv", index = False)
    

E, finalmente, a execução do código.

In [17]:
def main():
    
    df = read_excel(fp = "../data/CWTS Leiden Ranking 2019.xlsx", 
                    sheet = "Results")
    
    df = df.join( df.apply( split_years, axis = 1 ) )
    
    # Cinco com maior total de artigos, Brasil
    extract_top_univs(df = df, 
                      country = "Brazil",
                      period_end = 2017,
                      n_items = 5,
                      ordering_col = 'impact_P', 
                      filename = 'impact_P')

               
    # Cinco com maior total de artigos top 10%, Brasil
    extract_top_univs(df = df, 
                      country = "Brazil",
                      period_end = 2017,
                      n_items = 5,
                      ordering_col = 'P_top10', 
                      filename = 'P_top10')
    
    # Cinco com maior percentual de artigos entre os top 10% globais
    extract_top_univs(df = df, 
                  country = "Brazil",
                  period_end = 2017,
                  n_items = 5,
                  ordering_col = 'PP_top10', 
                  filename = 'PP_top10')
    
                   
    # Todas as universidades brasileiras – total de artigos top 10%
    extract_top_univs(df = df, 
                      country = "Brazil",
                      period_end = 2017,
                      n_items = 50,
                      ordering_col = 'P_top10', 
                      filename = 'todas_unis_P_top10',
                      color = 'random')

    # Soma total de artigos do Brasil
    get_aggregate(df = df, 
                  country = "Brazil",
                  period_end = 2017,
                  measure = 'impact_P', 
                  opp = 'sum',
                  filename = 'soma_brasil')

    # Coordenadas XY para o scatter plot
    get_xy(df, x_col = "PP_top10", y_col = "P_top10", period_end = 2017, country = "Brazil", filename = "coordenadas_scatter")

In [18]:
if __name__ == "__main__":
    main()